# Tarea Árboles de desición

#### Equipo:

**Aide Jazmín González Cruz**

El objetivo de la última tarea consistirá en modificar el Jupyter notebook que se proporcionó con el código CaRT para que además de resolver problemas de clasificación sirva para resolver problemas de regresión. Probarlo con el set de datos de bateadores que se utilizó en clase para que puedan validar que está funcionando bien. Compararlo contra el resultado en R. Para ello habrá que importar el data frame de R en python y convertirlo en una lista de listas.

In [66]:
# Importando librerias necesarias
import pandas as pd
import numpy as np
import statistics

Cargando datos

In [122]:
hitters = pd.read_csv('Hitters.csv')
hitters.shape

(263, 21)

In [126]:
hitters.columns

Index(['AtBat', 'Hits', 'HmRun', 'Runs', 'RBI', 'Walks', 'Years', 'CAtBat',
       'CHits', 'CHmRun', 'CRuns', 'CRBI', 'CWalks', 'PutOuts', 'Assists',
       'Errors', 'Salary'],
      dtype='object')

In [124]:
# Delete first column (names)
hitters.drop(hitters.columns[[0]], axis=1, inplace=True)
hitters.head(5)

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,Salary
0,315,81,7,24,38,39,14,3449,835,69,321,414,375,632,43,10,6.163315
1,479,130,18,66,72,76,3,1624,457,63,224,266,263,880,82,14,6.173786
2,496,141,20,65,78,37,11,5628,1575,225,828,838,354,200,11,3,6.214608
3,321,87,10,39,42,30,2,396,101,12,48,46,33,805,40,4,4.516339
4,594,169,4,74,51,35,11,4408,1133,19,501,336,194,282,421,25,6.620073


Si se usan todas las variables incluyendo las categóricas, estás serían algunos inputs en las funciones

In [108]:
num_col_salary = 18
# Column labels.
# These are used only to print the tree.
header = ["AtBat","Hits","HmRun","Runs","RBI","Walks","Years",    
          "CAtBat","CHits","CHmRun","CRuns","CRBI","CWalks","League" ,  
          "Division","PutOuts","Assists","Errors","Salary","NewLeague",
          "salary_c"]

In [123]:
# DEjando sólo las variable númericas
hitters.drop(['League'], axis=1, inplace=True)
hitters.drop(['Division'], axis=1, inplace=True)
hitters.drop(['NewLeague'], axis=1, inplace=True)
hitters

,Unnamed: 0,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,Salary
0,-Alan Ashby,315,81,7,24,38,39,14,3449,835,69,321,414,375,632,43,10,6.163315
1,-Alvin Davis,479,130,18,66,72,76,3,1624,457,63,224,266,263,880,82,14,6.173786
2,-Andre Dawson,496,141,20,65,78,37,11,5628,1575,225,828,838,354,200,11,3,6.214608
3,-Andres Galarraga,321,87,10,39,42,30,2,396,101,12,48,46,33,805,40,4,4.516339
4,-Alfredo Griffin,594,169,4,74,51,35,11,4408,1133,19,501,336,194,282,421,25,6.620073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,-Willie McGee,497,127,7,65,48,37,5,2703,806,32,379,311,138,325,9,3,6.551080
259,-Willie Randolph,492,136,5,76,50,94,12,5511,1511,39,897,451,875,313,381,20,6.774224
260,-Wayne Tolleson,475,126,3,61,43,52,6,1700,433,7,217,93,146,37,113,7,5.953243
261,-Willie Upshaw,573,144,9,85,60,78,8,3198,857,97,470,420,332,1314,131,12,6.866933


In [125]:
quitar = ["AtBat","HmRun","Runs","RBI","Walks",  
          "CAtBat","CHits","CHmRun","CRuns","CRBI","CWalks",
          "PutOuts","Assists","Errors"]
hits_years = hitters.copy()
hits_years = pd.DataFrame(hits_years.drop(quitar, axis=1))
hits_years.head(5)

,Hits,Years,Salary
0,81,14,6.163315
1,130,3,6.173786
2,141,11,6.214608
3,87,2,4.516339
4,169,11,6.620073


Pasando a Lista de Listas

- Para todas las variable númericas

In [128]:
hitters_lol = hitters.values.tolist()

- Sólo usando `hits` y `years`

In [130]:
hits_years_lol = hits_years.values.tolist()

### Funciones profesor

In [152]:
# Funciones profesor
def unique_vals(rows, col):
    """Find the unique values for a column in a dataset."""
    return set([row[col] for row in rows])
    #set regresa un conjunto({}) y regresa valores únicos

# Para la regresión mayor a menor y los promedios de datos subsecuentes y vecinos
# pe entre 2 y 3 -> 2.5 

def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

class Question:
    """A Question is used to partition a dataset.
    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))
    
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

# Para clasificación
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.
    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

# Caída del Gini
def info_gain(left, right, current_uncertainty):
    """Information Gain.
    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

### Pruebas profesor para clasificación

In [153]:
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]
header = ["color", "diameter", "label"]

In [154]:

#######
# Demo:
unique_vals(training_data, 0)
# unique_vals(training_data, 1)
#######

{'Green', 'Red', 'Yellow'}

In [156]:
class_counts(training_data)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}

In [157]:
#######
# Demo:
is_numeric(7)
#is_numeric("Red")
#######

True

In [162]:
#######
# Demo:
# Let's write a question for a numeric attribute
q = Question(0, 'Green')
q

Is color == Green?

In [160]:
# How about one for a categorical attribute
Question(1, 3)

Is diameter >= 3?

In [163]:
# Let's pick an example from the training set...
example = training_data[0]
# ... and see if it matches the question
q.match(example) # this will be true, since the first example is Green.
#######

True

In [164]:
#######
# Demo:
# Let's partition the training data based on whether rows are Red.
true_rows, false_rows = partition(training_data, Question(0, 'Red'))
# This will contain all the 'Red' rows.
true_rows

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

In [166]:
# This will contain everything else.
false_rows
#######

[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

In [167]:
#######
# Demo:
# Let's look at some example to understand how Gini Impurity works.
#
# First, we'll look at a dataset with no mixing.
no_mixing = [['Apple'],
             ['Apple']]
# this will return 0 % de clasificar de manera erronea las cosas
gini(no_mixing)

0.0

In [168]:
# Now, we'll look at dataset with a 50:50 apples:oranges ratio
some_mixing = [['Apple'],
               ['Orange']]
# this will return 0.5 - meaning, there's a 50% chance of misclassifying
# a random example we draw from the dataset.
# 50 % de clasificar de manera erronea las cosas
gini(some_mixing)

0.5

In [169]:
# Now, we'll look at a dataset with many different labels
lots_of_mixing = [['Apple'],
                  ['Orange'],
                  ['Grape'],
                  ['Grapefruit'],
                  ['Blueberry']]
# This will return 0.8
gini(lots_of_mixing)
#######

0.7999999999999998

In [172]:
#caida
#######
# Demo:
# Calculate the uncertainy of our training data.
current_uncertainty = gini(training_data)
current_uncertainty

0.6399999999999999

In [173]:

# How much information do we gain by partioning on 'Green'?
true_rows, false_rows = partition(training_data, Question(0, 'Green'))
info_gain(true_rows, false_rows, current_uncertainty)

0.1399999999999999

In [174]:
# What about if we partioned on 'Red' instead?
true_rows, false_rows = partition(training_data, Question(0,'Red'))
info_gain(true_rows, false_rows, current_uncertainty)

0.37333333333333324

In [175]:
# It looks like we learned more using 'Red' (0.37), than 'Green' (0.14).
# Why? Look at the different splits that result, and see which one
# looks more 'unmixed' to you.
true_rows, false_rows = partition(training_data, Question(0,'Red'))

# Here, the true_rows contain only 'Grapes'.
true_rows

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

In [176]:
# And the false rows contain two types of fruit. Not too bad.
false_rows

[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

In [177]:
# On the other hand, partitioning by Green doesn't help so much.
true_rows, false_rows = partition(training_data, Question(0,'Green'))

# We've isolated one apple in the true rows.
true_rows

[['Green', 3, 'Apple']]

In [178]:
# But, the false-rows are badly mixed up.
false_rows
#######

[['Yellow', 3, 'Apple'],
 ['Red', 1, 'Grape'],
 ['Red', 1, 'Grape'],
 ['Yellow', 3, 'Lemon']]

In [179]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [181]:
#######
# Demo:
# Find the best question to ask first for our toy dataset.
best_gain, best_question = find_best_split(training_data)
best_question
# FYI: is color == Red is just as good. See the note in the code above
# where I used '>='.
#######

Is diameter >= 3?

In [182]:
# Clase hoja, guarda los valores de las etiquetas
# En el caso de regresión aquí van los promedios o determinar promedios
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [183]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [184]:
# Cosntuye árbol, termina cuando la ganancia es 0, y ya no se pudo subdividir y se 
# crea una hoja, si no es cero genera 2 subconjuntos para crear 2 ramas
def build_tree(rows):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [185]:
# Impresión del árbol
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [188]:
my_tree = build_tree(training_data)

In [189]:
print_tree(my_tree)

Is diameter >= 3?
--> True:
  Is color == Yellow?
  --> True:
    Predict {'Apple': 1, 'Lemon': 1}
  --> False:
    Predict {'Apple': 1}
--> False:
  Predict {'Grape': 2}


## Funciones realizadas para variables continúas

In [190]:
# Funciones adicionales creadas

#Creando promedios entre valores
def promedio(lista):
    #print(lista)
    prom = []
    for i in range(len(lista)-1):
        prom.append((lista[i]+lista[i+1])/2)
    return prom

# Extrae una columna de una lista de listas dada
def Extract(lst, col_num): 
    return [item[col_num] for item in lst] 

# Para regresión (Error error cuadrado medio para cortes en tarea)
def rss(prom, col, salary):
    """
    Calcula la suma de residuos al cuadrado (RSS)
    """
    #Saca medias
    mean_ant_div = []
    mean_post_div = []
    for div in range(len(prom)):
        #print(div)
        ant_div = []
        post_div = []
        for i in range(len(col)):
            #print("" + str(col[i]))
            #col == years
            #print("years:" + str(col[i]))
            #print("div: "+ str(prom[div]))
            if (col[i] < prom[div]):#izq
                ant_div.append(salary[i])
                #print("izq:" + str(ant_div))
            else:#der
                post_div.append(salary[i])
                #print("der:" + str(post_div))
                
        #print(ant_div)
        #print(post_div)
        mean_ant_div.append(statistics.mean(ant_div))
        mean_post_div.append(statistics.mean(post_div))
        
    #print(mean_ant_div)
    #print(mean_post_div)
    
    
    # RSS
    rss_arr = []
    #print("prom" + str(prom))    
    #RSS = suma (y-yhat)²    
    for div in range(len(prom)):
        _rss = 0.0
        for i in range(len(col)):
            #print("RSS")
            #print("col : " + str(col[i]))
            #print("prom : " + str(prom[div]))
            if (col[i] < prom[div]):
                _rss = _rss + (salary[i] - mean_ant_div[div])**2                
            else:
                _rss = _rss + (salary[i] - mean_post_div[div])**2
        #print("_rss" + str(_rss))
        rss_arr.append(_rss)
        
    #print("rss: " + str(rss_arr))  
    error_min = min(rss_arr)
    error_max = max(rss_arr)
    valor_corte = prom[rss_arr.index(min(rss_arr))]
    #print("min: " + str(rss_arr.index(min(rss_arr))))
    #print("min: " + str(prom[rss_arr.index(min(rss_arr))]))
    return error_min, error_max, valor_corte

#Aquí se juntan las funciones anteriores para correr cada columna
def errores(lol, num_col_salary):
    errores_min = []
    errores_max = []
    values_c = []
    error_min = 0
    error_max = 0
    valor = 0
    #print("lol: "+str(len(lol)))
    for i in range(len(lol[0])-1):
        #print("lol" + str(lol[i]))
        col = Extract(lol, i)
        #print("col" + str(col))
        if is_numeric(col[0]):
            #print("col" + str(col))
            Split_col = unique_vals(lol,i)
            Split_col = list(Split_col)
            Split_col = sorted(Split_col)
            prom=promedio(Split_col)
            if (len(prom)>0):
                Salary = Extract(lol, num_col_salary)
                error_min, error_max, valor = rss(prom, col, Salary)
                errores_min.append(error_min)
                errores_max.append(error_max)
                values_c.append(valor)
            
    # Minímo valor        
    min_error = min(errores_min)
    min_val_index = errores_min.index(min(errores_min))
    #print("min_val_index: " + str(min_val_index))
    # Valor de corte del mínimo error
    corte_val = values_c[min_val_index]
    #print("corte_val: " + str(corte_val))
    # Indice de la columna con el menor valor
    #column_index = lol[0][min_val_index]
    #print("column_index: " + str(column_index))
    #column_name = header[column_index]
    column_name = header[min_val_index]
    return min_error, min_val_index, corte_val

In [191]:
def build_tree_2(rows, poda):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """
    
    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    ###gain, question = find_best_split(rows)
    if(len(rows) > (poda-1)):
        min_error, num_col, c_value  = min_error, name_col, c_value = errores(rows, num_col_salary)
        gain = min_error
        question = Question(num_col, c_value)
    else:
        return Leaf(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0 or len(rows) < poda:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)
    
    
    # Recursively build the true branch.
    true_branch = build_tree_2(true_rows, poda)

    # Recursively build the false branch.
    false_branch = build_tree_2(false_rows, poda)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [192]:
# Impresión del árbol
def print_tree2(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        #print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree2(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree2(node.false_branch, spacing + "  ")

### Pruebas

In [193]:
num_col_salary = 16
#hitters_lol
# Column labels sin categóricas
# These are used only to print the tree.
header = ["AtBat","Hits","HmRun","Runs","RBI","Walks","Years",    
          "CAtBat","CHits","CHmRun","CRuns","CRBI","CWalks",  
          "PutOuts","Assists","Errors","Salary"]

In [194]:
#Con el campo años
Split_years = unique_vals(hitters_lol,6)
Split_years = list(Split_years)
Split_years = sorted(Split_years)
#Split_years
# Probando promedios entre valores
prom = promedio(Split_years)
prom
Salary = Extract(hitters_lol, num_col_salary)
#Salary
# El mejor dato de la división
error_min, error_max, val_corte = rss(prom, Extract(hitters_lol,6),Salary)
print("error min: " + str(error_min) + ", valor de corte: " + str(val_corte))

error min: 115.05847519916081, valor de corte: 4.5


In [195]:
#Con el campo hits
Split_hits = unique_vals(hitters_lol,1)
Split_hits = list(Split_hits)
Split_hits = sorted(Split_hits)
#Split_years
# Probando promedios entre valores
prom = promedio(Split_hits)
prom
Salary = Extract(hitters_lol, num_col_salary)
#Salary
# El mejor dato de la división
error_min, error_max, val_corte = rss(prom, Extract(hitters_lol,1),Salary)
print("error min: " + str(error_min) + ", valor de corte: " + str(val_corte))

error min: 160.9715298124983, valor de corte: 117.5


In [196]:
true_rows, false_rows = partition(hitters_lol, Question(6, 4.5))

In [197]:
unique_vals(true_rows,6)

{5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 24.0}

In [198]:
unique_vals(false_rows,6)

{1.0, 2.0, 3.0, 4.0}

In [199]:
#Con el campo hits para la rama derecha
Split_hits = unique_vals(true_rows,1)
Split_hits = list(Split_hits)
Split_hits = sorted(Split_hits)
#
Salary = Extract(true_rows, num_col_salary)
Salary
#Split_hits
prom = promedio(Split_hits)
#prom
rss(prom, Extract(true_rows,1),Salary)

(48.97678249450081, 72.6595406653801, 117.5)

In [200]:
#Con el campo years para la rama derecha
Split_years = unique_vals(true_rows,6)
Split_years = list(Split_years)
Split_years = sorted(Split_years)
#
Salary = Extract(true_rows, num_col_salary)
Salary
#Split_hits
prom = promedio(Split_years)
#prom
rss(prom, Extract(true_rows,6),Salary)

(69.87701889391077, 72.6800485480704, 6.5)

In [201]:
#Con el campo years para la rama izquierda
Split_years = unique_vals(false_rows,6)
Split_years = list(Split_years)
Split_years = sorted(Split_years)
#
Salary = Extract(false_rows, num_col_salary)
Salary
#Split_hits
prom = promedio(Split_years)
#prom
rss(prom, Extract(false_rows,6),Salary)

(33.14306582442718, 38.98184820582178, 3.5)

In [202]:
#Con el campo years para la rama izquierda
Split_hits = unique_vals(false_rows,1)
Split_hits = list(Split_hits)
Split_hits = sorted(Split_hits)
#
Salary = Extract(false_rows, num_col_salary)
Salary
#Split_hits
prom = promedio(Split_hits)
#prom
rss(prom, Extract(false_rows,1),Salary)

(33.0145870443409, 42.33821620845311, 15.5)

In [203]:
num_cols = len(hitters_lol[0]) 
num_cols

17

In [205]:
min_error, name_col, c_value = errores(hitters_lol, num_col_salary)
min_error

89.29612127664447

In [206]:
name_col

7

In [207]:
header[name_col]

'CAtBat'

In [208]:
c_value

1452.0

In [209]:
Question(7, c_value)

Is CAtBat >= 1452.0?

#### Probando para todo el conjunto con las variables Hits y Years

In [210]:
num_col_salary = 2
header = ["Hits","Years","Salary"]
poda = 80
my_tree = build_tree_2(hits_years_lol, poda)
print_tree2(my_tree)

Is Years >= 4.5?
--> True:
  Is Hits >= 117.5?
  --> True:
    Is Hits >= 208.5?
    --> True:
    --> False:
      Is Hits >= 185.0?
      --> True:
      --> False:
  --> False:
    Is Years >= 6.5?
    --> True:
    --> False:
--> False:
  Is Hits >= 15.5?
  --> True:
    Is Years >= 3.5?
    --> True:
    --> False:
  --> False:


#### Probando para variables continuas

In [211]:
num_col_salary = 16
# Column labels.
# These are used only to print the tree.
header = ["AtBat","Hits","HmRun","Runs","RBI","Walks","Years",    
          "CAtBat","CHits","CHmRun","CRuns","CRBI","CWalks",
          "PutOuts","Assists","Errors","Salary"]

In [212]:
poda = 80
my_tree = build_tree_2(hitters_lol, poda)
print_tree2(my_tree)

Is CAtBat >= 1452.0?
--> True:
  Is Hits >= 117.5?
  --> True:
    Is CRBI >= 273.0?
    --> True:
    --> False:
  --> False:
--> False:
  Is CHits >= 182.0?
  --> True:
  --> False:


## No sé si esto también se va a implementar *

In [50]:
# Es un predict
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [52]:
#######
# Demo:
# The tree predicts the 1st row of our
# training data is an apple with confidence 1.
classify(hitters_lol[0], my_tree)
#######

{'Apple': 1}

In [53]:
# Puede regresar probabilidades del nod hoja
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [54]:
#######
# Demo:
# Printing that a bit nicer
print_leaf(classify(hitters_lol[0], my_tree))
#######

{'Apple': '100%'}

In [55]:
#######
# Demo:
# On the second example, the confidence is lower
print_leaf(classify(hitters_lol[1], my_tree))
#######

{'Apple': '50%', 'Lemon': '50%'}

In [56]:

# Evaluate
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [57]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: Apple. Predicted: {'Apple': '100%'}
Actual: Apple. Predicted: {'Apple': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Apple': '50%', 'Lemon': '50%'}
